In [1]:
!pip install datasets evaluate rouge_score
!pip install --upgrade transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 983.0 kB/s eta 0:00:00 0:00:01m
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=479cbdd63d32d846f8f031cf96ba4a17bb2f5beccf97085e5b28cb4ec904dbbc
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 26.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 57.5 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.2 MB/s eta 0:00:00
  Attempting uninstall: safetensors
    Found existing installation: safetensors 0.3.1
    Uninstalling safetensors-0.3.1:
      Successfully uninstalle

# Hyperparameters

In [2]:

# Set hyperparameters
BATCH_SIZE = 4
NUM_TRAIN_EPOCHS = 3
LEARNING_RATE = 3e-5
WEIGHT_DECAY = 0.01
MAX_SOURCE_LENGTH = 512
MAX_TARGET_LENGTH = 64
TRAINING_DATASET_SIZE = 100
TESTING_DATASET_SIZE = 10

DATASET_PATH = "/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv"
TEST_DATASET_PATH = "/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv"

#CHANGE THIS BEFORE ANY HYPERPARAMETER CHANGE !!!!!!!!!!!!
OUTPUT_DIR_CHECKPOINT = "/kaggle/working/model_testing_10k_6_14_2023_1"

# Load BillSum dataset
Loading the BillSum dataset with TRAINING_DATASET_SIZE and TESTING_DATASET_SIZE

In [3]:
from datasets import load_dataset

# Load the dataset
billsum = load_dataset("csv", data_files={"train": DATASET_PATH, "test": TEST_DATASET_PATH})

billsum["train"] = billsum["train"].select(range(TRAINING_DATASET_SIZE))
billsum["test"] = billsum["test"].select(range(TESTING_DATASET_SIZE))

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)
/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d31c9e8e00e39157/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Split the dataset into a train and test set with the [train_test_split](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.train_test_split) method:

In [4]:
#billsum = billsum.train_test_split(test_size=0.2)

In billsum dataset, There are two fields that you'll want to use:

- `clean_text`: the preprocessed text of the bill which'll be the input to the model.
- `summary`: a condensed version of `text` which'll be the model target.

In [5]:
billsum["train"][0]

{'id': '0001d1afc246a7964130f43ae940af6bc6c57f01',
 'article': "By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A . State Immunization Program Manager Molly Howell says the risk is low, but officials feel it's important to alert people to the possible exposure. The diocese announced on Monday that Bishop John Folda is taking time off after being diagnosed with hepatitis A. The diocese says he contracted the infection through con

# Preprocess
The next step is to load a `bart` tokenizer to process `text` and `summary`:

In [6]:
import os
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
checkpoint_dir = OUTPUT_DIR_CHECKPOINT

checkpoint = "facebook/bart-large-cnn"

if os.path.exists(checkpoint_dir):
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_dir)
    print("Using checkpoint model: ", checkpoint_dir)
else:
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

The preprocessing function you want to create needs to:

1. Prefix the input with a prompt so `facebook/bart-large-cnn` knows this is a summarization task.
2. Use the keyword `text_target` argument when tokenizing labels.
3. Truncate sequences to be no longer than the maximum length set by the `max_length` parameter.

In [11]:
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=MAX_SOURCE_LENGTH, truncation=True)

    labels = tokenizer(text_target=examples["highlights"], max_length=MAX_TARGET_LENGTH, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

Apply the preprocessing function over the entire dataset method and speed up the `map` function by setting `batched=True` to process multiple elements of the dataset at once:

In [12]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Creating a batch of examples using `DataCollatorForSeq2Seq` which dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [13]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

# Writing The Evaluate function
Loaded the `ROUGE` metric:

In [14]:
import evaluate
rouge = evaluate.load("rouge")

Then use `compute_metrics` for the bill sum predictions and labels to `compute` to calculate the ROUGE metric:

In [15]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

# Train
Training the `facebook\bart-large-cnn` model using AutoModelForSeq2SeqLM which loads the pretrained model

In [16]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

At this point, only three steps remain:

1. Using `Seq2SeqTrainingArguments`, we can configure the hyperparameter for the model, At the end of each step, the `trainer` will evaluate the ROUGE metric and save the training checkpoint.
2. Pass the training arguments to `Seq2SeqTrainer` along with the model, dataset, tokenizer, data collator, and `compute_metrics` function.
3. Call `train()` and `save_model()` to train and save the model.

In [17]:
from transformers.trainer_callback import EarlyStoppingCallback
os.environ["WANDB_DISABLED"] = "true"

if not os.path.exists(checkpoint_dir):
    # Set up the training arguments
    training_args = Seq2SeqTrainingArguments(
        output_dir=checkpoint_dir,
        evaluation_strategy="steps",  # Change evaluation strategy to "steps"
        learning_rate=LEARNING_RATE,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        weight_decay=WEIGHT_DECAY,
        save_total_limit=1,
        num_train_epochs=NUM_TRAIN_EPOCHS,
        predict_with_generate=True,
        fp16=True,
        logging_steps=1,
        load_best_model_at_end=True
    )

    # Set up the trainer with early stopping
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_billsum["train"],
        eval_dataset=tokenized_billsum["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    # Train the model
    trainer.train()
    trainer.save_model(checkpoint_dir)
else:
    print("Checkpoint already exists. Skipping training.")

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.884300,2.424106,0.492500,0.262000,0.354200,0.418000,78.600000
2,2.328900,2.179036,0.486700,0.272400,0.348400,0.433400,78.100000
3,2.102600,2.092962,0.480000,0.265100,0.358300,0.415700,71.600000
4,2.688800,2.021408,0.487700,0.252500,0.359700,0.413200,72.100000
5,2.098000,1.967266,0.494500,0.249900,0.346900,0.409700,69.700000
6,1.765800,1.934508,0.477800,0.249700,0.351800,0.416200,69.900000
7,1.713000,1.895838,0.459900,0.244500,0.346600,0.402700,72.200000
8,1.580300,1.859030,0.468500,0.250800,0.358000,0.410700,72.600000
9,2.311800,1.825379,0.467800,0.241200,0.338000,0.408200,71.700000
10,1.691300,1.787329,0.444300,0.216100,0.331500,0.390600,68.400000


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [18]:
# Evaluation on training dataset
train_metrics = trainer.evaluate(eval_dataset=tokenized_billsum["train"])
print("Training ROUGE Scores:")
print("ROUGE-1:", train_metrics["eval_rouge1"])
print("ROUGE-2:", train_metrics["eval_rouge2"])
print("ROUGE-L:", train_metrics["eval_rougeL"])

# Evaluation on testing dataset
test_metrics = trainer.evaluate(eval_dataset=tokenized_billsum["test"])
print("Testing ROUGE Scores:")
print("ROUGE-1:", test_metrics["eval_rouge1"])
print("ROUGE-2:", test_metrics["eval_rouge2"])
print("ROUGE-L:", test_metrics["eval_rougeL"])

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Training ROUGE Scores:
ROUGE-1: 0.5352
ROUGE-2: 0.3666
ROUGE-L: 0.4415
Testing ROUGE Scores:
ROUGE-1: 0.4588
ROUGE-2: 0.2138
ROUGE-L: 0.2971


# Inference
For `facebook/bart-large-cnn`, you need to prefix your input depending on the task you're working on. For summarization you should prefix your input as shown below:

In [19]:
text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."

Using `summarize_text`, we can tokenize the text and use the `input_ids` for generating summary from the model with `model.generate()`:

In [20]:
import torch

def summarize_text(input_text):

    text = "summarize: " + input_text

    inputs = tokenizer(text, max_length=MAX_SOURCE_LENGTH, truncation=True, return_tensors="pt").input_ids
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    # Move the model to the GPU
    model.to(device)

    # Move the inputs tensor to the GPU
    inputs = inputs.to(device)

    # Generate outputs
    outputs = model.generate(inputs, max_new_tokens=MAX_TARGET_LENGTH, do_sample=False)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return summary


Using the first sample of `train` dataset

In [22]:
print(billsum["train"][0]["highlights"])

Bishop John Folda, of North Dakota, is taking time off after being diagnosed .
He contracted the infection through contaminated food in Italy .
Church members in Fargo, Grand Forks and Jamestown could have been exposed .


In [23]:
summarize_text(billsum["train"][0]["article"])

'Bishop John Folda of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus.\nThe state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion.\nB'

Using the first sample of `test` dataset

In [24]:
print(billsum["test"][0]["highlights"])

Experts question if  packed out planes are putting passengers at risk .
U.S consumer advisory group says minimum space must be stipulated .
Safety tests conducted on planes with more leg room than airlines offer .


In [39]:
summarize_text(billsum["train"][0]["article"])

'Bishop John Folda of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus.\nThe state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion.\nB'

In [27]:
!pip install rogue

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rogue: filename=rogue-0.0.2-py3-none-any.whl size=7231 sha256=218ca957bc20199e3751199748a9e0d3ce4c3c554f28257b87adfe390b8e617e
  Stored in directory: /root/.cache/pip/wheels/11/db/ee/a44bd5d88bd4903f31c1f677a28ac79e354555bca48332f232
Successfully built rogue


In [38]:
from rouge import Rouge
def calculate_rouge_score(ref_summ, gen_summ):
    rouge = Rouge()

    # Calculate ROUGE scores
    scores = rouge.get_scores(gen_summ, ref_summ)

    # Extract ROUGE-1, ROUGE-2, and ROUGE-L scores
    rouge_1_score = scores[0]["rouge-1"]["f"]
    rouge_2_score = scores[0]["rouge-2"]["f"]
    rouge_l_score = scores[0]["rouge-l"]["f"]
    return rouge_1_score, rouge_2_score, rouge_l_score

ModuleNotFoundError: No module named 'rouge'

In [31]:
text="Sally Forrest, an actress-dancer who graced the silver screen throughout the '40s and '50s in MGM musicals and films such as the 1956 noir While the City Sleeps died on March 15 at her home in Beverly Hills, California. Forrest, whose birth name was Katherine Feeney, was 86 and had long battled cancer. Her publicist, Judith Goffin, announced the news Thursday. Scroll down for video . Actress: Sally Forrest was in the 1951 Ida Lupino-directed film 'Hard, Fast and Beautiful' (left) and the 1956 Fritz Lang movie 'While the City Sleeps' A San Diego native, Forrest became a protege of Hollywood trailblazer Ida Lupino, who cast her in starring roles in films including the critical and commercial success Not Wanted, Never Fear and Hard, Fast and Beautiful. Some of Forrest's other film credits included Bannerline, Son of Sinbad, and Excuse My Dust, according to her iMDBÂ page. The page also indicates Forrest was in multiple Climax! and Rawhide television episodes. Forrest appeared as herself in an episode of The Ed Sullivan Show and three episodes of The Dinah Shore Chevy Show, her iMDB page says. She also starred in a Broadway production of The Seven Year Itch. City News Service reported that other stage credits included As You Like It, No, No, Nanette and Damn Yankees. Forrest married writer-producer Milo Frank in 1951. He died in 2004. She is survived by her niece, Sharon Durham, and nephews, Michael and Mark Feeney. Career: A San Diego native, Forrest became a protege of Hollywood trailblazer Ida Lupino, who cast her in starring roles in films ."

In [33]:
summarize_text(text)

'Forrest, whose birth name was Katherine Feeney, was 86 and had long battled cancer.\nForrest was a protege of Hollywood trailblazer Ida Lupino, who cast her in films including the critical and commercial success Not Wanted, Never Fear and Hard, Fast and Beautiful.'

In [34]:
text2="A middle-school teacher in China has inked hundreds of sketches that are beyond be-leaf. Politics teacher Wang Lian, 35,  has created 1000 stunning ink drawings covering subjects as varied as cartoon characters and landscapes to animals, birds according toÂ theÂ People's Daily Online. The intricate scribbles on leaves feature Wang's favourite sites across the city of Nanjing, which include the Presidential Palace, Yangtze River Bridge, the ancient Jiming Temple and the Qinhuai River. Natural canvas: Artist and teacher Wang Lian has done hundreds of drawings, like this temple, on leaves she collects in the park and on the streets . Delicate: She uses an ink pen to gently draw the local scenes and buildings on the dried out leaves . 'Although teaching politics is my job, drawing is my passion and hobby,' said Wang. 'I first tried drawing on leaves about 10 years ago and fell in love with it as an art form immediately. 'It's like drawing on very old parchment paper, you have to be really careful that you don't damage the leaf because it is very fragile and this helps focus your attention and abilities.' Wang started giving the drawings away onÂ Christmas Eve in 2012 when her junior high school son came home saying he wanted to prepare some gifts for his classmates. Being an avid painter, Wang decided to give her son's friends unique presents of gingko leaf paintings. Wang loves gingko leaves and will often pick them up along Gingko Avenue, near to her school, in Nanjing in east China's Jiangsu province. Every autumn she collects about 2,000 leaves from the ground to ensure she has enough to cover spoils too. Intricate: Teacher Wang has drawn hundreds of local scenes on leaves she has collected from the park . Hobby: The artist collects leaves every autumn and dries them out so she can sketch these impressive building scenes . 'The colour and shape of gingko leaves are particularly beautiful,' she said. 'I need to collect around 2000 leaves because this will include losses'. She takes them home where she then presses them between the pages of books. 'Luckily, I have quite a lot of books and I try to use old ones or ones that I've already read so I don't end up with nothing to read.' Once they are dried, she carefully takes each one and using an ink fountain pen creates her masterpieces. She said: 'Some people are into capturing beauty through photography, but for me, a digitalised image just isn't the same. New leaf:Â Politics teacher Wang Lian has drawn hundreds of doodles on leaves for the last 10 years . 'By drawing what I see I become far more a part of the process and part of the final piece. 'One day I hope to be able to put my collection on display, but for now it's really just for my own pleasure.' Wang's leaf paintings are turned into bookmarks, postcards and sometimes even given as gifts to her her students so she can share the beauty of leaf paintings. But locals who have had the luck of being able to see Wang's art have been gobsmacked. Local art collector On Hao, 58, said: 'These are truly remarkable and beautiful creations. 'She has so much talent she is wasted in teaching.'"

In [35]:
summarize_text(text2)

'Wang Lian, 35, has created 1000 stunning ink drawings on leaves in Nanjing, China.\nThe political teacher collects leaves every autumn and dries them out so she can draw them.\nShe uses an ink fountain pen to create intricate designs on the delicate leaves.\nHer favourite sites'

In [36]:
text3="A man convicted of killing the father and sister of his former girlfriend in a fiery attack on the family's Southern California home was sentenced to death on Tuesday. Iftekhar Murtaza, 30, was sentenced for the murders of Jay Dhanak, 56, and his daughter Karishma, 20, in May 2007, the Orange County district attorney's office said. Murtaza was convicted in December 2013 of killing the pair in an attempt to reunite with his then-18-year-old ex-girlfriend Shayona Dhanak. She had ended their relationship citing her Hindu family's opposition to her dating a Muslim. To be executed: Iftekhar Murtaza, 30, was sentenced to death Tuesday for the May 21, 2007 murders of his ex-girlfriend's father and sister and the attempted murder of her mother . Authorities said Murtaza and a friend torched the family's Anaheim Hills home and kidnapped and killed Dhanak's father and sister, leaving their stabbed bodies burning in a park 2 miles from Dhanak's dorm room at the University of California, Irvine. Dhanak's mother, Leela, survived the attack. She was stabbed and left unconscious on a neighbor's lawn. Murtaza was interviewed by police several days later and arrested at a Phoenix airport with a ticket to his native Bangladesh and more than $11,000 in cash. Jurors recommended that Murtaza be sentenced to death for the crimes. Attack: Murtaza torched his ex-girlfriend's family's Orange County home after they broke-up, believing the murders of her family would reunited them . Religious differences: Murtaza dated Shayona Dhanak when she was 18 in 2007. She broke up with him when her Hindu parents allegedly told her they would stop paying her college tuition if she continued to date the Muslim man . Two of his friends were also sentenced to life in prison for the murders, but one of them, Vitaliy Krasnoperov, recently had his conviction overturned on appeal. Authorities said Krasnoperov hatched the plot to kill the Dhanaks with Murtaza and tried to help him hire a hit man. They said another friend, Charles Murphy Jr., helped Murtaza carry out the killings after Dhanak said she planned to go on a date with someone else. During the trial, Murtaza testified that he told many people he wanted to kill the Dhanaks because he was distraught over the breakup, but he said he didn't mean it literally. Didn't do it alone: Two of Murtaza's friends have been convicted in connection to the killings . Killer: Leela Dhanak testified how Iftekhar Murtaza, seen in this August photo, murdered her husband and elder daughter in a failed attempt to win over her younger daughter . Bloodbath: Autopsy reports showed Jayprakash Dhanak (left) suffered 29 stab wounds to his body, while a pathologist testified that Karishma Dhanak (right) was alive when her throat was slit and her body set alight ."

In [37]:
summarize_text(text3)

'Iftekhar Murtaza, 30, was sentenced to death for the 2007 murders of Jay Dhanak, 56, and his daughter Karishma, 20, in May 2007.\nMurtaza was convicted in December 2013 of killing the pair in an attempt to reunite with his then-'